In [39]:
from backtesting import Backtest, Strategy
from backtesting.test import GOOG
from backtesting.lib import crossover
from market_data import get_market_data
import talib as tl
import pandas as pd
import numpy as np



In [3]:
df = pd.DataFrame(get_market_data())
df_4h = df.resample('4H').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last'
}).dropna()
# # token_id: str = 'harrypotterobamasonic10in'
# # intervals: int = 30
# # currency: str = "usd"
df_4h.head(10)


,Open,High,Low,Close
date,,,,
2024-07-16 12:00:00,0.135121,0.137704,0.131512,0.134365
2024-07-16 16:00:00,0.134176,0.138078,0.132313,0.138078
2024-07-16 20:00:00,0.138409,0.146245,0.138409,0.146245
2024-07-17 00:00:00,0.146697,0.148938,0.143899,0.148393
2024-07-17 04:00:00,0.148119,0.152227,0.145687,0.146270
2024-07-17 08:00:00,0.146090,0.150087,0.146090,0.148922
2024-07-17 12:00:00,0.148849,0.151686,0.142101,0.143546
2024-07-17 16:00:00,0.143415,0.144236,0.140687,0.143446
2024-07-17 20:00:00,0.143462,0.143509,0.136381,0.137154


In [40]:
class Tin(Strategy):
    initial_investment = 100_000
    pieces = 5
    piece_value = initial_investment / pieces
    current_investment = piece_value
    profit_target = 1.5
    loss_threshold = 0.7

    def init(self):
        self.price = self.data.Close

    def next(self):
        current_price = self.price[-1]
        size = int(self.piece_value / current_price)

        if not self.position:
            self.buy(size=size)
            buy_date = self.data.index[-1].date()
            # print(f"Buy order executed at: {buy_date}")

        for trade in self.trades:
            if trade.pl >= self.piece_value * (self.profit_target - 1):
                trade.close()
                close_date = self.data.index[-1].date()
                # print(f"Position closed at: {close_date}")
            elif trade.pl <= -self.piece_value * (1 - self.loss_threshold):
                trade.close()
                close_date = self.data.index[-1].date()
                # print(f"Position closed at: {close_date}")

In [23]:
bt = Backtest(GOOG, Tin, cash=20_000, commission=.002)
stats = bt.run()

In [41]:
trades = stats['_trades']
print(trades)

   Size  EntryBar  ExitBar  EntryPrice  ExitPrice         PnL  ReturnPct  \
0    24         2      718   110.97150     525.00  9936.68400   3.730944   
1    24       719     1074   525.92976     262.51 -6322.07424  -0.500865   
2    24      1075     2024   269.79852     684.24  9946.59552   1.536115   
3    24      2025     2147   685.36800     797.80  2698.36800   0.164046   

   EntryTime   ExitTime  Duration  
0 2004-08-23 2007-06-27 1038 days  
1 2007-06-28 2008-11-21  512 days  
2 2008-11-24 2012-08-30 1375 days  
3 2012-08-31 2013-03-01  182 days  


In [42]:
print(stats)
bt.plot()

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    99.90689
Equity Final [$]                  36259.57328
Equity Peak [$]                   36476.77328
Return [%]                          81.297866
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                    7.229487
Volatility (Ann.) [%]               13.453883
Sharpe Ratio                         0.537353
Sortino Ratio                        0.844607
Calmar Ratio                         0.215165
Max. Drawdown [%]                   -33.59972
Avg. Drawdown [%]                   -2.031719
Max. Drawdown Duration     1786 days 00:00:00
Avg. Drawdown Duration       56 days 00:00:00
# Trades                                    4
Win Rate [%]                             75.0
Best Trade [%]                     373.094443
Worst Trade [%]                    -50.086491
Avg. Trade [%]                    

C:\Users\James\AppData\Roaming\Python\Python39\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\James\AppData\Roaming\Python\Python39\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\James\AppData\Roaming\Python\Python39\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\James\AppData\Roaming\Python\Python39\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p2282', ...)

In [43]:
print(stats[6]) # Return [%]

81.29786639999999


In [36]:
profit_target = tuple(float(f"{x:.2f}") for x in np.arange(1.05, 1.51, 0.05))
loss_threshold = tuple(float(f"{x:.2f}") for x in np.arange(0.5, 0.951, 0.05))

result = bt.optimize(
    pieces=[2, 4, 5, 10],
    profit_target=profit_target,
    loss_threshold=loss_threshold,
    maximize='Return [%]'
)

C:\Users\James\AppData\Roaming\Python\Python39\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 400 configurations.
  output = _optimize_grid()


In [38]:
print(result[6])
print(result._strategy)

87.79529040000001
Tin(pieces=2,profit_target=1.2,loss_threshold=0.65)


In [37]:
tuple(float(f"{x:.2f}") for x in np.arange(0.5, 0.951, 0.05))

(0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95)